In [1]:
# Import necessary libraries;
import numpy as np
from numpy import mean
from numpy import std


import pandas as pd

import matplotlib.pyplot as plt
from scipy import stats

# use seaborn plotting defaults
import seaborn as sns; sns.set_style('white')

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)


from mpl_toolkits.mplot3d import Axes3D
from numpy import genfromtxt
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.datasets import load_digits, make_blobs
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, plot_confusion_matrix
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler 
from sklearn.preprocessing import RobustScaler, Normalizer, QuantileTransformer, PowerTransformer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score
from sklearn import tree
import plotly.express as px

In [2]:
leagueData = pd.read_csv('LeagueMatches.csv')
del leagueData['Unnamed: 0']
del leagueData['matchId']
leagueData.head()

,blue_win,blueGold,blueMinionsKilled,blueJungleMinionsKilled,blueAvgLevel,redGold,redMinionsKilled,redJungleMinionsKilled,redAvgLevel,blueChampKills,blueHeraldKills,blueDragonKills,blueTowersDestroyed,redChampKills,redHeraldKills,redDragonKills,redTowersDestroyed
0,0,24575.0,349.0,89.0,8.6,25856.0,346.0,80.0,9.2,6.0,1.0,0.0,1.0,12.0,2.0,0.0,1.0
1,0,27210.0,290.0,36.0,9.0,28765.0,294.0,92.0,9.4,20.0,0.0,0.0,0.0,19.0,2.0,0.0,0.0
2,1,32048.0,346.0,92.0,9.4,25305.0,293.0,84.0,9.4,17.0,3.0,0.0,0.0,11.0,0.0,0.0,4.0
3,0,20261.0,223.0,60.0,8.2,30429.0,356.0,107.0,9.4,7.0,0.0,0.0,3.0,16.0,3.0,0.0,0.0
4,1,30217.0,376.0,110.0,9.8,23889.0,334.0,60.0,8.8,16.0,3.0,0.0,0.0,8.0,0.0,0.0,2.0


In [3]:
X = leagueData.iloc[:,1:16].values
y = leagueData.iloc[:,0].values
#To be used for validation purposes with best model for each trial 5000 train and 45000 test
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.90, random_state=42)

# RANDOM FORREST TRIAL ONE ON LEAGUE OF LEGENDS  DATASET

In [4]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                

    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_1 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL ONE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    3.7s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    4.8s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    5.6s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:   10.1s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:   12.1s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   14.2s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   16.3s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   18.1s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   19.8s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   22.6s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   24.9s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   27.1s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL ONE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   47.2s finished


In [5]:
TRIAL_1.cv_results_

{'mean_fit_time': array([0.56218333, 1.14978867, 2.30628324, 0.68428884, 1.32634068,
        2.6980206 , 0.75264745, 1.529215  , 3.01549325, 0.78987927,
        1.51260076, 3.01068907, 0.74023662, 1.4497468 , 2.92051148,
        0.85133228, 1.68985319, 3.20695815, 0.92479515, 1.77182355,
        3.51151996, 0.94751468, 1.86340241, 3.71689663, 0.56778855,
        1.10064659, 2.16586256, 0.61192632, 1.26108437, 2.57141118,
        0.70760894, 1.43253198, 2.80170937, 0.85393462, 1.56594653,
        3.02199898, 0.68208699, 1.37258053, 2.82803221, 0.80649362,
        1.64261255, 3.18523922, 0.91708851, 1.77943048, 3.64683585,
        0.97393742, 1.96148701, 3.92367454, 0.58720498, 1.15409265,
        2.27665792, 0.67347932, 1.44344144, 2.84814935, 0.75935283,
        1.45695314, 3.08345165, 0.82621055, 1.5516345 , 3.01989751,
        0.69810023, 1.37478232, 2.77008219, 0.90858126, 1.74590168,
        3.2827229 , 0.90938215, 1.79094024, 3.554457  , 1.04169598,
        2.03154731, 3.1403007 ]

In [6]:
print("---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_neg_log_loss']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL ONE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 400}

In [7]:
print("---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_f1_micro']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL ONE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [8]:
print("---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------")
TRIAL_1.cv_results_['rank_test_roc_auc_ovo']
TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL ONE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [9]:
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_1.cv_results_['params'][ np.argmin(TRIAL_1.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best NEG LOG LOSS hyperparameters :0.7813913122916001
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best F1 hyperparameters :0.7791988306764719
------------------------------------------------------------------------------------------------
Accuracy of Trial 1 RF using best ROC_AUC hyperparameters :0.7802037180834056
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL TWO ON LEAGUE OF LEGENDS  DATASET

In [10]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
                

    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_2 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL TWO RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.2s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    3.0s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    4.2s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    7.6s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   11.5s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   15.7s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   17.3s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   20.0s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   24.5s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL TWO RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   43.8s finished


In [11]:
TRIAL_2.cv_results_

{'mean_fit_time': array([0.52144823, 1.16440129, 2.22161078, 0.67037668, 1.40500822,
        2.70002193, 0.74744263, 1.50069094, 2.88027763, 0.81380062,
        1.60958414, 3.02500134, 0.74433975, 1.47136459, 2.73605332,
        0.7905798 , 1.54392786, 3.1752305 , 0.88786364, 1.87541275,
        3.69838085, 0.99635701, 2.07638583, 3.79426317, 0.5828012 ,
        1.11105576, 2.22321177, 0.64265256, 1.27409577, 2.46702142,
        0.68558969, 1.3458571 , 2.73285041, 0.75074558, 1.46836247,
        2.91220446, 0.68608966, 1.32463908, 2.67730227, 0.80088878,
        1.62709889, 3.16582251, 0.95832396, 1.78873811, 3.71579542,
        1.03448906, 1.95428052, 3.88013635, 0.5967133 , 1.12606854,
        2.16005759, 0.63864927, 1.28610568, 2.54428835, 0.73042822,
        1.45945501, 2.9321218 , 0.78267312, 1.51760488, 3.01158986,
        0.70550685, 1.37548285, 2.69711967, 0.80218997, 1.58416219,
        3.1866405 , 0.89807224, 1.72658477, 3.51662426, 1.02748361,
        1.94607358, 3.06123266]

In [12]:
print("---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_neg_log_loss']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL TWO NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [13]:
print("---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_f1_micro']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL TWO F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [14]:
print("---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------")
TRIAL_2.cv_results_['rank_test_roc_auc_ovo']
TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL TWO ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 400}

In [15]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_2.cv_results_['params'][ np.argmin(TRIAL_2.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best NEG LOG LOSS hyperparameters :0.780021011282145
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best F1 hyperparameters :0.7798839811811995
------------------------------------------------------------------------------------------------
Accuracy of Trial 2 RF using best ROC_AUC hyperparameters :0.7802265564335632
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL THREE ON LEAGUE OF LEGENDS  DATASET

In [16]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_3 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL THREE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.9s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    4.1s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    7.5s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    9.3s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   11.3s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   13.4s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   15.4s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   16.9s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   19.6s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   21.7s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   24.0s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL THREE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   42.4s finished


In [17]:
TRIAL_3.cv_results_

{'mean_fit_time': array([0.53355899, 1.06421528, 2.17486992, 0.60862336, 1.34445605,
        2.60664163, 0.71581564, 1.39319801, 2.84114327, 0.77937002,
        1.59637313, 3.15010896, 0.75374794, 1.46215758, 2.72784595,
        0.83892155, 1.61498895, 3.16051807, 0.91838956, 1.79914708,
        3.60960417, 0.93340249, 1.91584778, 3.80487208, 0.56488581,
        1.08433275, 2.18888216, 0.64215221, 1.25628023, 2.42558594,
        0.67508039, 1.34625754, 2.68771148, 0.71761718, 1.42882867,
        2.87397151, 0.67798271, 1.3269412 , 2.63166289, 0.77606726,
        1.59747381, 3.10677166, 0.91648817, 1.76051426, 3.4674819 ,
        0.92849851, 1.83317661, 3.67415977, 0.53836293, 1.06081209,
        2.09870501, 0.61092496, 1.24076662, 2.44560332, 0.69249535,
        1.34295502, 2.66869493, 0.70820889, 1.41321568, 2.83163519,
        0.65876641, 1.29841666, 2.62545781, 0.7816721 , 1.54713044,
        3.06523614, 0.86474361, 1.71647615, 3.3905158 , 0.9284986 ,
        1.80104904, 2.94383163]

In [18]:
print("---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_neg_log_loss']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL THREE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [19]:
print("---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_f1_micro']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL THREE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 100}

In [20]:
print("---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------")
TRIAL_3.cv_results_['rank_test_roc_auc_ovo']
TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL THREE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [21]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_3.cv_results_['params'][ np.argmin(TRIAL_3.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))


------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best NEG LOG LOSS hyperparameters :0.7813227972411273
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best F1 hyperparameters :0.7811172520897091
------------------------------------------------------------------------------------------------
Accuracy of Trial 3 RF using best ROC_AUC hyperparameters :0.7810715753893939


# RANDOM FOREST TRIAL FOUR ON LEAGUE OF LEGENDS  DATASET

In [22]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_4 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FOUR RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    6.9s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    8.6s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   10.8s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   12.8s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   14.6s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   20.7s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FOUR RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   41.3s finished


In [23]:
TRIAL_4.cv_results_

{'mean_fit_time': array([0.51704459, 1.03639107, 2.09690289, 0.61092515, 1.23506222,
        2.43299212, 0.67287855, 1.32844238, 2.64827757, 0.70520649,
        1.39139667, 2.81882405, 0.65996757, 1.45084753, 2.69411678,
        0.8078949 , 1.54572911, 3.0847527 , 0.8625421 , 1.69005356,
        3.41343522, 0.90607934, 1.82526941, 3.6335247 , 0.5306561 ,
        1.06301384, 2.11011438, 0.60572076, 1.21884804, 2.43599458,
        0.67778277, 1.35266314, 2.67429957, 0.70110254, 1.40100479,
        2.79170051, 0.66627278, 1.31403012, 2.64577508, 0.77776871,
        1.5410253 , 3.07274246, 0.86043968, 1.71347337, 3.41403604,
        0.91188402, 1.81275907, 3.62962117, 0.53075657, 1.07072072,
        2.12212481, 0.60972424, 1.23115869, 2.42028151, 0.68048515,
        1.33795047, 2.65868635, 0.70780845, 1.40701008, 2.82202687,
        0.66407104, 1.3133296 , 2.64637551, 0.76986194, 1.54612966,
        3.07264237, 0.85493526, 1.70226388, 3.32415905, 0.9132854 ,
        1.7772284 , 2.84744892]

In [24]:
print("---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_neg_log_loss']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FOUR NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 13,
 'classifier__max_features': 'auto',
 'classifier__n_estimators': 200}

In [25]:
print("---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_f1_micro']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FOUR F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [26]:
print("---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------")
TRIAL_4.cv_results_['rank_test_roc_auc_ovo']
TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FOUR ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 200}

In [27]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_4.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best NEG LOG LOSS hyperparameters :0.7794500525282053
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best F1 hyperparameters :0.7811172520897091
------------------------------------------------------------------------------------------------
Accuracy of Trial 4 RF using best ROC_AUC hyperparameters :0.780843191887818
------------------------------------------------------------------------------------------------


# RANDOM FOREST TRIAL FIVE ON LEAGUE OF LEGENDS   DATASET

In [28]:
import warnings
warnings.filterwarnings('ignore')

ITERATIONS = 1
for i in range(1):
    #For reach "trial", randomly select 5000 data points
    random5000DataPoints = leagueData.sample(n = 5000, replace = True)
    #From that 5000, create x set and y set
    xSet = random5000DataPoints.iloc[:,1:17].values
    ySet = random5000DataPoints.iloc[:,0].values
    # Run GridSearch to find best hyperparameters per 5000 data points 
    pipe = Pipeline([('std', StandardScaler()),('classifier', RandomForestClassifier(n_jobs = -1))])

    search_space = [
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['auto'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                 
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': [ 'sqrt'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']},
                
                {
                
                'classifier': [RandomForestClassifier()],
                'classifier__n_estimators': [100,200,400],
                'classifier__max_features': ['log2'],
                'classifier__max_depth' : [7,9,11,13],
                'classifier__criterion' :['gini', 'entropy']
                
                }
                    ]
                
    #'precision_micro','roc_auc_ovo','jaccard','neg_log_loss'] JACCARD DOES NOT WORK FOR NON BINARY?
    runHyperParamSearch = GridSearchCV(pipe, search_space, cv=StratifiedKFold(n_splits=5), 
                          scoring=['recall_micro','f1_micro','roc_auc_ovo','neg_log_loss'], 
                          refit=False, verbose=10, n_jobs = -1)
    TRIAL_5 = runHyperParamSearch.fit(xSet, ySet)
    print("---------------------------------TRIAL FIVE RESULTS ---------------------------------------------")

Fitting 5 folds for each of 72 candidates, totalling 360 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.9s
[Parallel(n_jobs=-1)]: Done  29 tasks      | elapsed:    2.7s
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    3.8s
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed:    5.3s
[Parallel(n_jobs=-1)]: Done  66 tasks      | elapsed:    6.8s
[Parallel(n_jobs=-1)]: Done  81 tasks      | elapsed:    8.5s
[Parallel(n_jobs=-1)]: Done  96 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done 113 tasks      | elapsed:   12.7s
[Parallel(n_jobs=-1)]: Done 130 tasks      | elapsed:   14.5s
[Parallel(n_jobs=-1)]: Done 149 tasks      | elapsed:   16.0s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   18.8s
[Parallel(n_jobs=-1)]: Done 189 tasks      | elapsed:   20.6s
[Parallel(n_jobs=-1)]: Done 210 tasks      | elapsed:   23.1s
[Parallel(n_jobs=-1)]: Done 233 tasks      | elapsed:  

---------------------------------TRIAL FIVE RESULTS ---------------------------------------------


[Parallel(n_jobs=-1)]: Done 360 out of 360 | elapsed:   41.7s finished


In [29]:
TRIAL_5.cv_results_

{'mean_fit_time': array([0.51684427, 1.03929319, 2.07308249, 0.60582075, 1.21864753,
        2.42048125, 0.66617293, 1.31763334, 2.65318165, 0.70840907,
        1.39319811, 2.82052574, 0.65246091, 1.30372124, 2.6078424 ,
        0.78547525, 1.57215142, 3.1160799 , 0.86974773, 1.76241579,
        3.45847425, 0.94841552, 1.82927303, 3.65574393, 0.53125691,
        1.0729228 , 2.10531063, 0.61202612, 1.22705512, 2.42138271,
        0.67107668, 1.34425592, 2.68380823, 0.70760841, 1.4009182 ,
        2.81862378, 0.70360498, 1.3763835 , 2.68290691, 0.78017073,
        1.55693955, 3.05622811, 0.8788559 , 1.84999089, 3.44276047,
        0.9553216 , 1.91324511, 3.67175775, 0.53275857, 1.07502437,
        2.13523645, 0.62623873, 1.23256006, 2.46531982, 0.69359617,
        1.34405584, 2.70052247, 0.73172922, 1.4094121 , 2.81091728,
        0.65596371, 1.31483059, 2.65468307, 0.77416573, 1.63570652,
        3.10577097, 0.92189298, 1.70977063, 3.44115877, 0.98985114,
        1.89502935, 2.92911882]

In [30]:
print("---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_neg_log_loss']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]

---------------------------------TRIAL FIVE NEG LOG LOSS RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 400}

In [31]:
print("---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_f1_micro']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_f1_micro']) ]

---------------------------------TRIAL FIVE F1 RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'gini',
 'classifier__max_depth': 11,
 'classifier__max_features': 'sqrt',
 'classifier__n_estimators': 100}

In [32]:
print("---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------")
TRIAL_5.cv_results_['rank_test_roc_auc_ovo']
TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]

---------------------------------TRIAL FIVE ROC AUC RESULTS ---------------------------------------------


{'classifier': RandomForestClassifier(),
 'classifier__criterion': 'entropy',
 'classifier__max_depth': 13,
 'classifier__max_features': 'log2',
 'classifier__n_estimators': 400}

In [33]:

print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best NEG LOG LOSS results to train on test set, 
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_neg_log_loss']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train__neg_log_loss = pipe.predict(X_train)
y_pred_test_neg_log_loss = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_neg_log_loss)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best F1 results to train on test set, LR_Trial 1
p = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_4.cv_results_['rank_test_f1_micro']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_f1 = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best F1 hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_f1)))
print("------------------------------------------------------------------------------------------------")
# Using Best hyperparameters of best ROC_AUC results to train on test set, LR_Trial 1
p  = TRIAL_5.cv_results_['params'][ np.argmin(TRIAL_5.cv_results_['rank_test_roc_auc_ovo']) ]
pipe.set_params(**p)
pipe.fit(X_train,y_train)
y_pred_train_roc_auc = pipe.predict(X_train)
y_pred_test_roc_auc = pipe.predict(X_test)
print('Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :'
      + str(accuracy_score(y_test,y_pred_test_roc_auc)))
print("------------------------------------------------------------------------------------------------")


------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best NEG LOG LOSS hyperparameters :0.7814598273420728
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best F1 hyperparameters :0.7811629287900242
------------------------------------------------------------------------------------------------
Accuracy of Trial 5 RF using best ROC_AUC hyperparameters :0.7804777782852966
------------------------------------------------------------------------------------------------
